# pymongo数据库操作

### python执行mongo的shell命令

In [2]:
print(db.command("dbstats")) 
print(db.name)

### 创建数据库

In [ ]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='127.0.0.1', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'testdb')
db.command("updateUser", "arvin", pwd="arvin123")

# 创建管理员用户,密码和权限
db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])

### 检测mongodb服务是否可用

In [42]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

client = MongoClient()

try:
    # The ismaster command is cheap and does not require auth.
    client.admin.command('ismaster')
    print('Server is available')
except ConnectionFailure:
    print("Server is not available")

Server is available


### 连接mongodb(无auth验证)

无验证式连接数据库通常是在第一次连接数据库时使用到,此时没有建立数据库,没有配置用户角色和读写权限.

In [26]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='127.0.0.1', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'testdb')
db.command("updateUser", "arvin", pwd="arvin123")

# 创建管理员用户,密码和权限
db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])
# 创建普通用户,密码和只读权限
db.command("createUser", "xiaoming", pwd="xiaoming@123456.", roles=["read"])
# 创建普通用户,密码和只写权限
db.command("createUser", "xiaohong", pwd="xiaohong@123456.", roles=["write"])
# 修改用户密码
db.command("updateUser", "xiaoming", pwd="xiaoming@123.")
# 修改用户读取权限
db.command("updateUser", "xiaoming", roles=["readWrite"])

{'ok': 1.0}

### mongodb连接(带有auth验证)

pymongo是线程安全的,但是不是fork-safe. 当多进程使用同一个client实例会报错:

**解决办法:** 在连接mongo数据库时,设置关键字`connect=False`,即在mongodb实例化时不建立连接,等到有数据库操作才进行数据库连接.

In [ ]:
import pymongo

try:
    # Python 3.x
    from urllib.parse import quote_plus
except ImportError:
    # Python 2.x
    from urllib import quote_plus

MONGODB = {
    "user": "arvin",
    "passwd": "arvin@123456.",
    "host": "127.0.0.1:27017",
    "dbname": "testdb"
}

# 方式一
# 以类的方式创建mongodb的实例
class ConnectMongo(object):
    
        def __init__(self, MONGODB):
        self.client = MongoClient(
            'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                           MONGODB['passwd'],
                                           MONGODB['host'],
                                           MONGODB['dbname']),
            connect=False
        )
        self.mongodb = self.client[MONGODB['dbname']]

    def __del__(self):
        self.client.close()
        
# 方式二        
def connect_mongo(MONGODB):
    client = MongoClient(
        'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                       MONGODB['passwd'],
                                       MONGODB['host'],
                                       MONGODB['dbname']),
        connect=False)
    mongodb = client[MONGODB['dbname']]
    return mongodb

# 方式三: 处理特殊字符 
def connect_mongo(mongo_param):
    uri =  'mongodb://{}:{}@{}/{}'.format(quote_plus(mongo_param['user']),
                                          quote_plus(mongo_param['passwd']),
                                          mongo_param['host'],
                                          mongo_param['dbname'])
    try:
        client = pymongo.MongoClient(uri, connect=False)
        mongodb = client[mongo_param['dbname']]
        return mongodb
    except Exception as e:
        print(e)

**注意:** 
1. 方式二存在内存泄漏问题,因为创建的client没有在程序结束后close掉.
2. 当用户名或密码带有 ‘:’, ‘/’, ‘+’ 和 ‘@’ 这些字符时需要使用百分比编码,否则连接报错: `Username and password must be escaped according to RFC 3986, use urllib.parse.quote_plus().`

In [4]:
import pymongo

# 仅仅创建mongodb连接,但未验证,没有连接到具体的数据库,相当于打开一个shell
client = pymongo.MongoClient(host='192.168.1.141', port=27017)
# 创建数据库
db = pymongo.database.Database(client, 'goshawk_test')
# db.command("createUser", "Goshawk", pwd="yYZmrjJ8xauAe#t7", roles=["readWrite"])

# 创建管理员用户,密码和权限
# db.command("createUser", "admin", pwd="admin@123456.", roles=["root"])
# 创建普通用户,密码和读取权限
# db.command("createUser", "arvin", pwd="arvin@123456.", roles=["readWrite"])

### map-reduce

In [ ]:
from bson.code import Code

mapfunc = Code('''function(){emit(this.tel, {platform: this.platform,time:this.time})}''')
reducefunc = Code(
    '''function(key, values){var time = 0;var platform = "";values.forEach(function(doc){if(doc.time>time){time=doc.time;platform=doc.platform;}});return {platform: platform,time:time};}''')
finalfunc = Code('''function(key, val){val.tel = key;return val;}''')

movie.map_reduce(mapfunc, reducefunc, out="collection_name", query={
                 "tel": {"$regex": '^[0-9]+$'}, "time": {"$gte": 0}}, finalize=finalfunc)


# map_reduce
# map函数的作用是遍历集合，调用emit函数将集合中userId、nick字段以键值对的形式传递给reduce函数
mapper = Code("""
function(){
    emit(this.userId, this.nick);
}
""")

# reduce函数的作用对map函数传递过来的键值对进行处理, 
# 每个<key, values>键值对中，key是userId字段的值，values是具有相同userId的nick的数组。
# 由于我的程序中一个values的各个元素的值是相同的，所以没有对values进行遍历。
reducer = Code("""
function(key, values){
    var result = {nick: values[0]};
    return result;
}
""")

# 启动MapReduce, 将结果输出到seller_info这个集合中
result = collection.map_reduce(mapper, reducer, 'seller_info')

# 遍历seller_info集合，查看MapReduce结果
for doc in db['seller_info'].find():
    print doc

In [10]:
from itertools import zip_longest

data = b'Aug 30 14:53:39 ubuntu sshd[7636]: Accepted password for root from 192.168.187.1 port 7436 ssh2\n'
token = 'b2ddbd9212bf6c62cd658fab8c9ddd0a'
platform = 'linux'

def preprocess_data(data, platform):
    data_list = []
    lines = []
    if platform == 'windows':
        lines = data.split(b'\r\n')
    elif platform == 'linux':
        lines = data.split(b'\n')
    for line in lines:
        if line:
            try:
                str_line = line.decode()
            except:
                logging.error("data: {}, decode error: can't decode with utf-8.".format(line))
                str_line = str(line)[2:-1]
            data_list.append(str_line.strip())
    return data_list


def _handle_ssh_detail(line):
    str_time = line[:16]
    data = {'time': str_time}
    KEYS = ['status', 'user', 'IP', 'port']
    content = line[line.rfind(':')+1:].strip()
    details = content.split(' ')
    details = [item for item in details if details]
    print(details)
    details.pop(details.index('for')-1)
    details.pop(details.index('for'))
    details.pop(details.index('from'))
    details.pop(details.index('port')+2)
    details.pop(details.index('port'))
    for k, v in zip_longest(KEYS, details, fillvalue='_'):
        data[k] = v
    print(details)
    return data


def handle_ssh_data(data, platform, token):
    print(data)
    data_list = []
    lines = []
    for line in preprocess_data(data, platform):
        if 'sshd' in line and 'session' not in line:
            lines.append(line)

    for line in lines:
        data_item = _handle_ssh_detail(line)
        data_item['token'] = token
        data_list.append(data_item)
    print('ssh data: ', data_list)

handle_ssh_data(data, platform, token)

b'Aug 30 14:53:39 ubuntu sshd[7636]: Accepted password for root from 192.168.187.1 port 7436 ssh2\n'
['Accepted', 'password', 'for', 'root', 'from', '192.168.187.1', 'port', '7436', 'ssh2']
['Accepted', 'root', '192.168.187.1', '7436']
ssh data:  [{'time': 'Aug 30 14:53:39 ', 'status': 'Accepted', 'user': 'root', 'IP': '192.168.187.1', 'port': '7436', 'token': 'b2ddbd9212bf6c62cd658fab8c9ddd0a'}]


### in 和 not in (`$in $nin`) 判断数据是否存在某个集合

### 查询某个字段多个值的数据
db.collection.find({"A": {"$in": [1, 2, 3]}})

```
db.collection.find({"field":{$in:array}});


db.things.find({j:{$in: [2,4,6]}});
db.things.find({j:{$nin: [2,4,6]}});
```

### 取模运算($mod)

```
db.things.find("this.a%10 == 1")

db.things.find({a:{$mod:[10, 1]}})
```


4) 取模运算$mod

如下面的运算：
db.things.find( "this.a % 10 == 1")
可用$mod代替：

db.things.find( { a : { $mod : [ 10 , 1 ] } } )


5)  $all

$all和$in类似，但是他需要匹配条件内所有的值：

如有一个对象：

{ a: [ 1, 2, 3 ] }
下面这个条件是可以匹配的：

db.things.find( { a: { $all: [ 2, 3 ] } } );
但是下面这个条件就不行了：

db.things.find( { a: { $all: [ 2, 3, 4 ] } } );

6)  $size

$size是匹配数组内的元素数量的，如有一个对象：{a:["foo"]}，他只有一个元素：

下面的语句就可以匹配：db.things.find( { a : { $size: 1 } } );
官网上说不能用来匹配一个范围内的元素，如果想找$size<5之类的，他们建议创建一个字段来保存元素的数量。

You cannot use $size to find a range of sizes (for example: arrays with more than 1 element). If you need to query for a range, create an extra size field that you increment when you add elements.

7）$exists

$exists用来判断一个元素是否存在：

如：

db.things.find( { a : { $exists : true } } ); // 如果存在元素a,就返回
db.things.find( { a : { $exists : false } } ); // 如果不存在元素a，就返回

8)  $type

$type 基于 bson type来匹配一个元素的类型，像是按照类型ID来匹配，不过我没找到bson类型和id对照表。

db.things.find( { a : { $type : 2 } } ); // matches if a is a string
db.things.find( { a : { $type : 16 } } ); // matches if a is an int
9）正则表达式

mongo支持正则表达式，如：

db.customers.find( { name : /acme.*corp/i } ); // 后面的i的意思是区分大小写

10)  查询数据内的值

下面的查询是查询colors内red的记录，如果colors元素是一个数据,数据库将遍历这个数组的元素来查询。db.things.find( { colors : "red" } );

11) $elemMatch

如果对象有一个元素是数组，那么$elemMatch可以匹配内数组内的元素：

> t.find( { x : { $elemMatch : { a : 1, b : { $gt : 1 } } } } ) 
{ "_id" : ObjectId("4b5783300334000000000aa9"), 
"x" : [ { "a" : 1, "b" : 3 }, 7, { "b" : 99 }, { "a" : 11 } ]
}$elemMatch : { a : 1, b : { $gt : 1 } } 所有的条件都要匹配上才行。
注意，上面的语句和下面是不一样的。

> t.find( { "x.a" : 1, "x.b" : { $gt : 1 } } )
$elemMatch是匹配{ "a" : 1, "b" : 3 }，而后面一句是匹配{ "b" : 99 }, { "a" : 11 } 


12)  查询嵌入对象的值

db.postings.find( { "author.name" : "joe" } );
注意用法是author.name，用一个点就行了。更详细的可以看这个链接： dot notation

举个例子：

> db.blog.save({ title : "My First Post", author: {name : "Jane", id : 1}})
如果我们要查询 authors name 是Jane的, 我们可以这样：

> db.blog.findOne({"author.name" : "Jane"})
如果不用点，那就需要用下面这句才能匹配：

db.blog.findOne({"author" : {"name" : "Jane", "id" : 1}})
下面这句：

db.blog.findOne({"author" : {"name" : "Jane"}})
是不能匹配的，因为mongodb对于子对象，他是精确匹配。

13) 元操作符 $not 取反

如：

db.customers.find( { name : { $not : /acme.*corp/i } } );db.things.find( { a : { $not : { $mod : [ 10 , 1 ] } } } ); mongodb还有很多函数可以用，如排序，统计等，请参考原文。

mongodb目前没有或(or)操作符，只能用变通的办法代替，可以参考下面的链接：

http://www.mongodb.org/display/DOCS/OR+operations+in+query+expressions

### 数据导出与导入

In [ ]:
import subprocess
from pymongo import MongoClient

# 导出数据
def export_data(db, dst_path, param):
    for collection in db.collection_names():
        log.info("开始导出 {} 数据库 {} ...".format(db, collection))
        try:
            subprocess.check_call(["mongoexport",
                                   "-h", param["host"],
                                   "-u", param["user"],
                                   "-p", param["passwd"],
                                   "-d", db,
                                   "-c", collection,
                                   "-o", dst_path])
            log.info("导出 {} 数据成功".format(dst_path))
        except Exception as e:
            log.error("导出MongoDB数据出错,数据导出中止, Error: {}".format(e))
            

# 导入数据
# mongoimport -d 数据库名字 -c 集合名字 --type 文件类型 --file 文件路径  --upsert,
# 其中，--upsert字段的意思是以插入(insert)或者更新(update)的方式来导入数据。
# 注意：如果数据库存在数据，要导入最新的数据，则需要加--upsert选项，会更新数据，否则会报错(提示重复键错误收集)
def importdata(db_name, collection, file_name, datatype='json'):
    try:
        subprocess.Popen(['mongoimport', '-d', db_name, '-c', collection,
                         '--type', datatype, '--file', file_name, '--upsert'])
    except Exception as e:
        print("导入数据异常:{}".format(e))   
        

# 导入数据到远程服务器上
# mongoimport -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -c 表名 --upsert --drop 文件名的具体路径   (--drop当不需要删除源文件可不加)
def importdata(user, passwd, db_name, collection, file_name, host='127.0.0.1', port='27017', datatype='json'):
    try:
        subprocess.Popen(['mongoimport', '-h', host, '--port', port, 
                          '-u', user, '-p', passwd, '-d', db_name, '-c', collection,
                          '--type', datatype, '--file', file_name, '--upsert'])
    except Exception as e:
        print("导入数据异常:{}".format(e)) 
        

# 导入部分字段到collection中
# 命令格式：mongoimport -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -c 表名 --upsertFields 字段 --drop 文件名的具体路径 (--drop当不需要删除源文件可不加)

# 导入csv文件
# 命令格式：mongoimport -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -c 表名 --type 类型(csv) --headerline --upsert --drop 文件名的具体路径  

def exportdata(host, user, passwd, db_name, collection, dst):
    try:
        subprocess.Popen(['mongoexport', '-h', host, '-u', user,
                         '-p', passwd, '-d', db_name, '-c', collection, '-o', dst])
    except Exception as e:
        print("导出数据异常:{}".format(e))        

### 数据备份与还原

In [ ]:
# 备份命令: mongodump -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -o 文件存放路径
# 还原命令: mongorestore -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 --drop 文件存在路径,--drop是先删除所有的数据，再恢复，不需要删除可不加;

def dumpdata(host, user, passwd, db_name, collection, dst):
    try:
        subprocess.Popen(['mongodump', '-h', host, '-u', user,
                         '-p', passwd, '-d', db_name, '-c', collection, '-o', dst])
    except Exception as e:
        print("备份数据异常:{}".format(e))

def restoredata(host, user, passwd, db_name, collection, restorepath)
    try:
        subprocess.Popen(['mongoretore', '-h', host, '-u', user,
                          '-p', passwd, '-d', db_name, '-c', collection, restorepath])
    except Exception as e:
        print("还原数据异常:{}".format(e))